In [13]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore') 

import missingno as msno
from tqdm import tqdm_notebook

In [14]:
train_iden = pd.read_csv("train_identity.csv")
train_tran = pd.read_csv("train_transaction.csv")
test_iden = pd.read_csv("test_identity.csv")
test_tran = pd.read_csv("test_transaction.csv")

In [15]:
print (len (train_iden))
print (len (train_tran))
print (len (test_iden))
print (len (test_tran))

144233
590540
141907
506691


In [16]:
train_sample = train_tran[['isFraud','V280',
       'V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
sample = train_sample.fillna(1)

In [17]:
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import * 

In [19]:
import imblearn.under_sampling as usam
import imblearn.over_sampling as osam 
from sklearn.ensemble import RandomForestClassifier


X = sample.drop('isFraud', axis = 1)
Y = sample['isFraud']
feature_names = X.columns.tolist() # 컬럼을 리스트화
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

rus = usam.RandomUnderSampler()
undersampled_X, undersampled_Y = rus.fit_sample(Train_X, Train_Y)
model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=13).fit(undersampled_X, undersampled_Y)
predicted = model.predict(Test_X)
print (roc_auc_score(Test_Y, predicted))

0.7129737509147743


In [31]:
from sklearn.externals import joblib
joblib.dump(model, 'under_sampling_model.pkl')
model = joblib.load('under_sampling_model.pkl') 

In [37]:
test_sample = test_tran[['V280',
       'V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']].fillna(1)
len (test_sample)

506691

In [44]:
test_sample.head(1)

,V280,V284,V285,V286,V287,V290,V291,V292,V293,V294,V295,V297,V298,V299,V303,V304,V305,V306
0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
predicted = model.predict(test_sample)

In [49]:
dic = {'TransactionID':test_tran['TransactionID'],
       'isFraud':predicted}
submission = pd.DataFrame(dic)
len (submission)

506691

In [51]:
submission.to_csv('submission.csv', index = False)

In [52]:
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree

In [53]:
from sklearn.ensemble import VotingClassifier

In [58]:
rus = usam.RandomUnderSampler()
undersampled_X, undersampled_Y = rus.fit_sample(Train_X, Train_Y)
model = VotingClassifier(estimators=[('KNN',KNeighborsClassifier(n_neighbors = 9)),
                                              ('RBF',svm.SVC(probability = True,
                                                             kernel = 'rbf',C = 7,gamma = 0.1)),
                                              ('RFor',RandomForestClassifier(n_estimators = 100,
                                                                             random_state = 13)),
                                              ('LR',LogisticRegression(C = 0.1)),
                                              ('DT',DecisionTreeClassifier(max_depth = 5, 
                                                                           random_state = 13)),
                                              ('NB',GaussianNB())]
                                    ,voting = 'soft').fit(undersampled_X, undersampled_Y)
predicted = model.predict(Test_X)
print (roc_auc_score(Test_Y, predicted))

0.7186703449581164
